In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'IntroductionToNaturalLanguageProcessing/project/github_submission'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd /content/drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/IntroductionToNaturalLanguageProcessing/project/github_submission


In [ ]:
!pip install -U pip
!pip install accelerate==0.20.3
!pip install appdirs==1.4.4
!pip install bitsandbytes==0.37.2
!pip install datasets==2.10.1
!pip install fire==0.5.0
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torch==2.0.1
!pip install sentencepiece==0.1.97
!pip install tensorboardX==2.6
!pip install gradio==3.23.0

!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 20.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 20.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_sa

Data

In [ ]:
import pandas as pd

# get top 5 critics
df = pd.read_csv("data.csv")
critic_counts = df['critic_name'].value_counts()
top_5_critics = critic_counts.nlargest(5).index
df = df[df['critic_name'].isin(top_5_critics)]

In [ ]:
def get_reviews(df, critic_name, movie_title):
    critic_reviews = df[df['critic_name'] == critic_name]
    critic_reviews = critic_reviews[critic_reviews['movie_title'] != movie_title]
    reviews = critic_reviews['review_content'].sample(n=min(5, len(critic_reviews)), random_state=42)

    reviews = ['[REVIEW] ' + review for review in reviews]

    return reviews

In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

df, _ = train_test_split(df, test_size=0.4, random_state=42)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

def create_dataset_entry(df, row_dict):
    return {
        "movie_title" : row_dict['movie_title'],
        "movie_info": row_dict['movie_info'],
        "genres": row_dict.get("genres"),  # Use .get() to avoid KeyError if the key is not present
        "review_score": row_dict.get("review_score"),
        "critic_reviews_history" : get_reviews(df, row_dict['critic_name'], row_dict['movie_title']),
        "output": row_dict['review_content']
    }

train_dataset = [create_dataset_entry(train_data, row) for row in tqdm(train_data.to_dict(orient="records"), desc='Creating Train Dataset')]
test_dataset = [create_dataset_entry(test_data, row) for row in tqdm(test_data.to_dict(orient="records"), desc='Creating Test Dataset')]

instructions = [
    "Make new review",
    "Write a new review for {movie_title}",
    "You are a movie critic, using movie infos and review you have written before, write a new short review for movie {movie_title}"
]

train_test_datasets = {}
for i, instruction in enumerate(instructions):
    train_test_datasets[f'model{i+1}'] = {
        'train': [{**entry, "instruction": instruction.format(movie_title=entry['movie_title'])} for entry in tqdm(train_dataset, desc=f'Creating Training Model {i+1}')],
        'test': [{**entry, "instruction": instruction.format(movie_title=entry['movie_title'])} for entry in tqdm(test_dataset, desc=f'Creating Testing Model {i+1}')],
    }

Creating Testing Model 3: 100%|██████████| 3489/3489 [00:00<00:00, 492641.87it/s]


Hyperparameter

In [ ]:
# LEARNING_RATE = 1e-4
# TRAIN_STEPS = 5
# warmup_steps=1
# logging_steps=1
# eval_steps=1
# save_steps=5

# BATCH_SIZE = 128
# MICRO_BATCH_SIZE = 4
# GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE

In [ ]:
LEARNING_RATE = 3e-4
TRAIN_STEPS = 100
warmup_steps=30
logging_steps=10
eval_steps=20
save_steps=20

BATCH_SIZE = 128
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE

Alpaca setting

In [ ]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

In [ ]:
BASE_MODEL = "decapoda-research/llama-7b-hf"

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [ ]:
LORACONFIG = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

Prompt

In [ ]:
def generate_prompt1(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
[Movie_info] {data_point["movie_info"]}
{data_point["critic_reviews_history"]}
### Response:
{data_point["output"]}"""

def generate_prompt2(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
[Movie_info] {data_point["movie_info"]}
[Movie genres]{data_point["genres"]}
[Review score]{data_point["review_score"]}
[review history]{data_point["critic_reviews_history"]}
### Response:
{data_point["output"]}"""

def generate_prompt3(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
[Movie_info] {data_point["movie_info"]}
[Movie genres]{data_point["genres"]}
[Review score]{data_point["review_score"]}
[review history]{data_point["critic_reviews_history"]}
### Response:
{data_point["output"]}"""

# Store the prompt generation functions in a dictionary
prompt_generators = {
    "model1": generate_prompt1,
    "model2": generate_prompt2,
    "model3": generate_prompt3,
}


In [ ]:
def generate_test_prompt1(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
[Movie_info] {data_point["movie_info"]}
{data_point["critic_reviews_history"]}
### Response:
"""

def generate_test_prompt2(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
[Movie_info] {data_point["movie_info"]}
[Movie genres]{data_point["genres"]}
[Review score]{data_point["review_score"]}
[review history]{data_point["critic_reviews_history"]}
### Response:
"""

def generate_test_prompt3(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
[Movie_info] {data_point["movie_info"]}
[Movie genres]{data_point["genres"]}
[Review score]{data_point["review_score"]}
[review history]{data_point["critic_reviews_history"]}
### Response:
"""

# Store the prompt generation functions in a dictionary
test_prompt_generators = {
    "model1": generate_test_prompt1,
    "model2": generate_test_prompt2,
    "model3": generate_test_prompt3,
}


Tokenizer

In [ ]:
CUTOFF_LEN = 512

def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(model, data_point):
    full_prompt = prompt_generators[model](data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

Train Function

In [ ]:
def get_training_arguments(OUTPUT_DIR):
    training_arguments = transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=warmup_steps,
        max_steps=TRAIN_STEPS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=logging_steps,
        optim="adamw_torch",
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=eval_steps,
        save_steps=save_steps,
        output_dir=OUTPUT_DIR,
        save_total_limit=3,
        load_best_model_at_end=True,
        report_to="tensorboard"
    )

    return training_arguments

In [ ]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

Make Data Loader

In [ ]:
import json
import os
from datasets import load_dataset

def load_data(model, train_test_datasets):
    train_dataset_data = train_test_datasets[model]['train']
    test_dataset_data = train_test_datasets[model]['test']
    with open(f"generator_{model}_data.json", "w") as f:
        json.dump(train_dataset_data, f)

    data = load_dataset("json", data_files=f"generator_{model}_data.json")

    os.remove(f"generator_{model}_data.json")

    return data

Train Model

In [ ]:
def train_model(MODEL, BASE_MODEL, LORACONFIG):
    OUTPUT_DIR = f"experiments/generator/{MODEL}"
    data = load_data(MODEL, train_test_datasets)
    training_arguments = get_training_arguments(OUTPUT_DIR)

    train_val = data["train"].train_test_split(test_size=500, shuffle=True, seed=42)
    train_data = train_val["train"].shuffle().map(lambda x: generate_and_tokenize_prompt(MODEL, x))
    val_data = train_val["test"].shuffle().map(lambda x: generate_and_tokenize_prompt(MODEL, x))

    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, LORACONFIG)
    model.print_trainable_parameters()

    trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=training_arguments,
        data_collator=data_collator
    )
    model.config.use_cache = False
    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict()
        )
    ).__get__(model, type(model))
    model = torch.compile(model)
    trainer.train()
    model.save_pretrained(OUTPUT_DIR)

In [ ]:
# Train Model1
train_model("model1", "decapoda-research/llama-7b-hf", LORACONFIG)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b529d6d112612355/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/13452 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
20,2.490400,2.273063
40,1.684100,1.541366
60,1.020500,0.870511
80,0.725000,0.717563
100,0.703700,0.697004


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

In [ ]:
# Train Model2
train_model("model2", "decapoda-research/llama-7b-hf", LORACONFIG)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e2c6a377e4aef450/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/13452 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
20,2.393500,2.174501
40,1.567600,1.426408
60,0.957100,0.813045
80,0.679200,0.651597
100,0.623800,0.627975


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

In [22]:
# Train Model3
train_model("model3", "decapoda-research/llama-7b-hf", LORACONFIG)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-869ecce0be524b87/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/13452 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
20,2.414300,2.209157
40,1.529200,1.399772
60,0.955100,0.799427
80,0.645800,0.620967
100,0.590500,0.595862


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q